In [36]:
def extract_substring_checksum(text):
    start_index = text.find("checksum")
    
    if start_index != -1:
        start_index += 9  

    return text[start_index:len(text)-1]


In [37]:
import json
    
with open('/Users/maike/Desktop/fastjson/report.json', 'r') as file:
    data = json.load(file)

for killed in data["killed"]:
    killed["checksum"] = extract_substring_checksum(killed["processOutput"])

with open('/Users/maike/Desktop/fastjson/reports.json',"w") as f:
    json.dump(data,f)

In [14]:
def extract_substring_line(text):
    start_index = text.find("@@\n")
    
    if start_index != -1:
        start_index += 3  
    else:
        return "未找到 '@@\\n'"

    end_index = text.find("\n-")
    
    if end_index == -1:
        return "未找到 '\\n-'"

    return text[start_index:end_index]


In [35]:
import re
import json

REPO = "boltdb"

with open("/Users/maike/go/src/github.com/boltdb/bolt/db.go", 'r') as file:
    lines = file.readlines()
    
with open('/Users/maike/go/src/github.com/boltdb/bolt/reports.json', 'r') as file:
    data = json.load(file)

function_pattern = re.compile(r'^func(?:\s+\(\s*\w+\s+\*?\w+\s*\))?(\s+\w+)')

m ={}
for killed in data["killed"]:
    match = re.search(r'-([0-9]+),', killed["diff"])

    number = int(match.group(1)) 
    
    sub = extract_substring(killed["diff"])
    number = number + sub.count('\n') + 1

    function_name = ""
    for i in range(number - 1, -1, -1):  #
        match = function_pattern.search(lines[i])
        if match:
            function_name = match.group(1).strip()  # 
            break
    
    if  function_name == "":
        continue
    
    if REPO == "boltdb":
        if function_name == "Error":
            continue
        if not function_name[0].isupper():
            continue
				
    m[killed["checksum"]] = function_name
    
with open("function_names.json","w") as f:
    json.dump(m,f)